- to unzip a zip file we use the `tarfile` module

In [1]:
import tarfile
# with tarfile.open(r'D:\Python machine learning\datasets\movie_review\aclImdb_v1.tar.gz') as tar:
#     tar.extractall()

- now we will load the data into a right formate

In [2]:
import pyprind 
import pandas as pd 

import os 


# #the change the `basepath` to the directory of the unzippped movie dataset
# basepath=r'D:\Python machine learning\chapter-8,Sentiment analysis using ml\aclImdb'

# labels={'pos':1,'neg':0}

# pbar=pyprind.ProgBar(50000)
# df=pd.DataFrame()

# for s in ('test','train'):
#     for l in ('pos','neg'):
        
#         path=os.path.join(basepath,s,l)

#         for file in sorted(os.listdir(path)):
#             with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
#                 txt=infile.read()

#             df=df.append([[txt,labels[l]]],ignore_index=True)
#             pbar.update()

# df.columns=['review','sentiment']

- converting the assembled data into csv

In [3]:
import numpy as np 
# np.random.seed(0)

# df=df.reindex(np.random.permutation(df.index))

# df.to_csv(r'D:\Python machine learning\datasets\movie_review\movie_data.csv',encoding='utf-8',index=False)


In [4]:
df=pd.read_csv(r'D:\Python machine learning\datasets\movie_review\movie_data.csv',encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [5]:
df.shape

(50000, 2)

- now we will convert the text data into bag of word using the `CountVectorizer` like converting the text data into numeric data

In [6]:
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer 

#example code of workiing
count=CountVectorizer()

docs=np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining the weather is sweet,'
    'and one and one is two',

])

bag=count.fit_transform(docs)

In [7]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]], dtype=int64)

In [8]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


- here we will use the `TfidfTransformer` that is used to perform 
`tfidf(term frequency inverse document frequency)`

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer 
tfidf=TfidfTransformer(
    use_idf=True,
    norm='l2',
    smooth_idf=True
)

np.set_printoptions(precision=2)

print(tfidf.fit_transform(count.transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


- now we will learn to `clean the text data` 

In [10]:
df.loc[0,'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

- now we will use the `regular expression` for parsing the text

In [11]:
import re 

def preprocessing(text):
    text=re.sub('<[^>]*>','',text)
    emoticons=re.findall('(?::|,|=)(?:-)?(?:\)|\(|D|P)',text)

    text=(re.sub('[\W]+',' ',text.lower()))+' '.join(emoticons).replace('-','')

    return text

In [12]:
preprocessing(df.loc[0,'review'][-50:])
preprocessing('</a>This :) is :( a test :-)!')

'this is a test :) :( :)'

In [13]:
df['review']=df['review'].apply(preprocessing)

- now we will split the text at the whitespace using the .split function

In [14]:
def tokenizer(text):
    return text.split(' ')


tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

- now we will apply the stemming technique for tokeniztion

In [15]:
from nltk.stem.porter import PorterStemmer 
porter=PorterStemmer()


def tokenizer_porter(text):
    return [porter.stem(word=w) for w in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

- now we will use the stopwords function to remove stop-words as it contains non useful words 

In [16]:
import nltk 
# nltk.download('stopwords')
from nltk.corpus import stopwords 
stop=stopwords.words('english')

[word for word in tokenizer_porter('a runner likes running and runs a lot')[-10:] if word not in stop]

['runner', 'like', 'run', 'run', 'lot']

- now we will split the dataframe

In [17]:
X_train=df.loc[:25000,'review'].values
y_train=df.loc[:25000,'review'].values
X_test=df.loc[25000:,'review'].values
y_test=df.loc[25000:,'review'].values

- now we will use the GridSearchCV to find the best parameters

In [18]:
# from sklearn.model_selection import GridSearchCV 
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression 
# from sklearn.feature_extraction.text import TfidfVectorizer 

# tfidf=TfidfVectorizer(strip_accents=None,
#                       lowercase=False,
#                       preprocessor=None)

# param_grid=[
#     {
#         'vect__ngram_range':[(1,1)],
#         'vect__stop_words':[stop,None],
#         'vect__tokenizer':[tokenizer,tokenizer_porter],
#         'clf__penalty':['l1','l2'],
#         'clf__C':[1.0,10.0,100.0],

#     },
#     {
#         'vect__ngram_range':[(1,1)],
#         'vect__stop_words':[stop,None],
#         'vect__tokenizer':[tokenizer,tokenizer_porter],
#         'vect__use_idf':[False],
#         'vect__norm':[None],
#         'clf__penalty':['l1','l2'],
#         'clf__C':[1.0,10.0,100.0],
#     }
# ]

# #final model
# lr_tfidf=Pipeline(
#     [
#         ['vect',tfidf],
#         ['clf',LogisticRegression(random_state=0,solver='liblinear')]
#     ]
# )

# #grid search model
# gs_lr_tfidf=GridSearchCV(estimator=lr_tfidf,param_grid=param_grid,cv=3,scoring='accuracy',verbose=2,n_jobs=2)

# gs_lr_tfidf.fit(X_train,y_train)


In [19]:
# print('best parameter set :%s'%(gs_lr_tfidf.best_params_))

- now we will perform out of core learning 

In [20]:
import numpy as np 
import re 
from nltk.corpus import stopwords
stop=stopwords.words('english')

def tokenizer(text):
    text=re.sub('<[^>]*>','',text)
    emoticons=re.findall('(?::|,|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub('[\W]+',' ',text.lower())+''.join(emoticons).replace('-','')

    tokenized=[w for w in text.split() if w not in stop]

    return tokenized 

- now we will define a generator function, that reads and returns one document at a time

In [21]:
def stream_docs(path):
    with open(path,'r',encoding='utf-8') as csv:
        next(csv) #skip the header

        for line in csv:
            text,label=line[:-3],int(line[-2])
            yield text,label #this is will not return the entier csv but only one line at
            # a time and when the next function is called it keeps returning one line until 
            #there is no more lines
            #this helps in saving the memory while loading  

In [22]:
tokenizer(next(stream_docs(path=r'D:\Python machine learning\datasets\movie_review\movie_data.csv'))[0])

['1974',
 'teenager',
 'martha',
 'moxley',
 'maggie',
 'grace',
 'moves',
 'high',
 'class',
 'area',
 'belle',
 'greenwich',
 'connecticut',
 'mischief',
 'night',
 'eve',
 'halloween',
 'murdered',
 'backyard',
 'house',
 'murder',
 'remained',
 'unsolved',
 'twenty',
 'two',
 'years',
 'later',
 'writer',
 'mark',
 'fuhrman',
 'christopher',
 'meloni',
 'former',
 'la',
 'detective',
 'fallen',
 'disgrace',
 'perjury',
 'j',
 'simpson',
 'trial',
 'moved',
 'idaho',
 'decides',
 'investigate',
 'case',
 'partner',
 'stephen',
 'weeks',
 'andrew',
 'mitchell',
 'purpose',
 'writing',
 'book',
 'locals',
 'squirm',
 'welcome',
 'support',
 'retired',
 'detective',
 'steve',
 'carroll',
 'robert',
 'forster',
 'charge',
 'investigation',
 '70',
 'discover',
 'criminal',
 'net',
 'power',
 'money',
 'cover',
 'murder',
 'murder',
 'greenwich',
 'good',
 'tv',
 'movie',
 'true',
 'story',
 'murder',
 'fifteen',
 'years',
 'old',
 'girl',
 'committed',
 'wealthy',
 'teenager',
 'whose',


- now we will create the get_minibatch function to get the docs

In [23]:
def get_minibatch(doc_streams,size):
    docs,y=[],[]

    try:
        for _ in range(size):
            text,label=next(doc_streams)
            docs.append(text)
            y.append(label)
    
    except StopIteration:
        return None,None 

    return docs,y


In [24]:
from sklearn.feature_extraction.text import HashingVectorizer 
from sklearn.linear_model import SGDClassifier 

vect=HashingVectorizer(
    decode_error='ignore',
    n_features=2**21,
    preprocessor=None,
    tokenizer=tokenizer 
)

clf=SGDClassifier(loss='log_loss',random_state=1)  #by setting the loss to log we are modifying the Stochastic Gradient Descent Classifier into LogisticRegression
doc_stream=stream_docs(path=r'D:\Python machine learning\datasets\movie_review\movie_data.csv')

- now we start the out of core learning

In [25]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


- assigning the the documents to test set

In [26]:
X_test,y_test=get_minibatch(doc_stream,size=5000)
X_test=vect.transform(X_test)
print('Accuracy %0.3f'%(clf.score(X_test,y_test)))

Accuracy 0.868


- now we will update the classifier with the test set 

In [27]:
clf=clf.partial_fit(X_test,y_test)

- now we will decompose the movie review into 10 categories using the subcategory of unsupervised `LatentDirichletAllocation` `LDA` 

In [28]:
import pandas as pd 
df=pd.read_csv(r'D:\Python machine learning\datasets\movie_review\movie_data.csv',encoding='utf-8')

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

count=CountVectorizer(stop_words='english',
                      max_df=.1,
                      max_features=5000)

X=count.fit_transform(df['review'].values)


- fitting the `LatentDirichletAllocation`

In [30]:
from sklearn.decomposition import LatentDirichletAllocation 

lda=LatentDirichletAllocation(n_components=10,
                              random_state=123,
                              learning_method='batch')

X_topics=lda.fit_transform(X)

In [31]:
lda.components_.shape

(10, 5000)

In [32]:
n_tops_words=7 
feature_name=count.get_feature_names_out()

for topic_idx,topic in enumerate(lda.components_):
    print('Topic %d'%(topic_idx+1))
    print(', '.join([feature_name[i]
                    for i in topic.argsort()[:-n_tops_words-1:-1]]))

Topic 1
worst, minutes, awful, script, stupid, terrible, money
Topic 2
family, mother, father, children, girl, loved, kids
Topic 3
american, war, dvd, music, tv, history, german
Topic 4
human, audience, cinema, art, sense, feel, viewer
Topic 5
police, guy, car, dead, murder, wife, goes
Topic 6
horror, house, sex, girl, woman, blood, gore
Topic 7
role, performance, comedy, actor, performances, plays, played
Topic 8
series, episode, war, episodes, tv, season, action
Topic 9
book, version, original, read, novel, disney, effects
Topic 10
action, fight, guy, guys, cool, fun, minutes


In [33]:
horror=X_topics[:,0].argsort()[::-1]

for iter_idx,movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:'%(iter_idx+1))
    print(df['review'][movie_idx][:300],'...')


Horror movie #1:
I went into this with my hopes up, by twenty minutes into the movie I couldn't have been more let down. Despite thinking that this would be another horribly bad remake, I kept my hopes high that maybe...just maybe someone would get it right this time around. Sadly, Prom Night is about on the same qu ...

Horror movie #2:
It used to be that video distributors like Sub Rosa and Brain Damage Films would release low-budget, shot-on-video horror films to a select market of gorehounds that ate them up with glee. That's acceptable to me, because you could see these movies from a mile away with their shoddy box art and chee ...

Horror movie #3:
It seems like anybody can make a movie nowadays. It's like all you need is a camera, a group of people to be your cast and crew, a script, and a little money and walla you have a movie. Problem is that talent isn't always part of this equation and often times these kind of low budget films turn out  ...


In [34]:
df['review'][X_topics[:,7].argsort()[::-1][0]]

'A strong pilot, this two-hour episode does an excellent job of setting up the characters and background for "Enterprise," the "prequel" to the original "Star Trek" series. It stumbles a few times into "Trek" convention and cliché--candy-colored space strippers never seem to go out of style, and I can already foresee snickering references to T\'pol as "Seven of Vulcan"--but the ensemble looks strong, the characters are well-drawn, and one can already see hints that this particular crew will have to be more resourceful, in different ways, than those of earlier (later?) series. Scott Bakula hits the right note as a captain with Kirk\'s brashness and daring but without his smugness and swagger, and I look forward to the ways in which the series will feature the engineer, weapons master and communications officer (not just a glorified phone operator anymore!) as supporting players. The writers seem to have picked up on the one big mistake made in "Star Trek: The Next Generation," "Deep Spa

In [35]:
X_topics.shape

(50000, 10)

In [36]:
[lda.components_[5].argsort()[::-1]]

[array([2198, 2208, 3958, ..., 3835, 1847, 1130], dtype=int64)]

In [37]:
len(count.vocabulary_)

5000

In [50]:
for i in lda.components_:
    print(feature_name[i.argsort()[::-1][0]])

worst
family
american
human
police
horror
role
series
book
action


In [56]:
df['review'][X_topics[::,0].argsort()[::-1][:3]]

16597    I went into this with my hopes up, by twenty m...
41862    It used to be that video distributors like Sub...
4498     It seems like anybody can make a movie nowaday...
Name: review, dtype: object

## Chapter -9

In [58]:
import pickle 
import os 
dest=os.path.join(r'D:\Python machine learning\chapter-9,Machine learning to web\movieclassifier','pkl_objects')

if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop,open(os.path.join(dest,'stopwords.pkl'),'wb'),protocol=4) #wb:- write binary

pickle.dump(clf,open(os.path.join(dest,'classifier.pkl'),'wb'),protocol=4)